## Read File containing times

In [1]:
import re

In [2]:
def convert_time(str_time):
    # Expect time with format 'Minutes:seconds'
    m, s = str_time.split(':')
    return float(m)*60+float(s)

In [3]:
def read_log_file(log_file):

    with open(log_file) as time_file:
        line = time_file.readline()

        list_logged_info = ["Command being timed", "User time (seconds)", "System time (seconds)", "Percent of CPU this job got", "Elapsed (wall clock) time (h:mm:ss or m:ss)", "Average shared text size (kbytes)", "Average unshared data size (kbytes)", "Average stack size (kbytes)", "Average total size (kbytes)", "Maximum resident set size (kbytes)", "Average resident set size (kbytes)", "Major (requiring I/O) page faults", "Minor (reclaiming a frame) page faults", "Voluntary context switches", "Involuntary context switches", "Swaps", "File system inputs", "File system outputs", "Socket messages sent", "Socket messages received", "Signals delivered", "Page size (bytes)", "Exit status"]

        all_targets = []
        number_of_lines = 0
        nb_info = len(list_logged_info)
        previous_info = nb_info - 1 # Initialize with previous info being the last element of list_logged_info
        skipped_lines = []


        while line:
            line = line.strip()
            #print line
            try:
                key, info = line.split(": ")
                # Did we find the expected info line (i.e. are we on the next list_logged_info line)
                expected_key = list_logged_info[(previous_info+1)%nb_info]
                if key != expected_key:
                    raise Exception('Found key: "%s". Expecting: "%s". Previous key: "%s\nCommand line: %s\nline #: %d"'
                                    %(key, expected_key, list_logged_info[previous_info], line, number_of_lines))
                previous_info = (previous_info + 1)%nb_info
                # Save info
                if key == 'Command being timed':
                    cmd = info
                elif key == 'Percent of CPU this job got':
                    percent_cpu = float(info.split('%')[0])
                elif key == 'Elapsed (wall clock) time (h:mm:ss or m:ss)':
                    # Safe check
                    if cmd is None:
                        raise Exception('cmd is None. Line: %s' %line)
                    if percent_cpu is None:
                        raise Exception('percent_cpu is None. Line: %s' %line)
                    all_targets.append((cmd, convert_time(info)*percent_cpu/100.0))
                    # Reset `cmd` and `percent_cpu` just in case
                    cmd = None
                    percent_cpu = None
                # Move to next line
                line = time_file.readline()
                number_of_lines += 1
            except ValueError as e:
                print "Skipping a line that has multiple ':' character"
                skipped_lines.append(line)
                # Find next command that corresponds to first key
                found_first_key = False
                while not found_first_key:
                    line = time_file.readline()
                    number_of_lines += 1
                    line = line.strip()
                    split_line = line.split(": ")
                    if split_line[0] == list_logged_info[0]:
                        found_first_key = True
        print("Number of lines: %d" % number_of_lines)
        print("Number of skipped lines: %s" % len(skipped_lines))
        print("Number of targets: %s" % len(all_targets))
        
        return all_targets, skipped_lines

In [4]:
def TargetTime(all_targets,total_make_time, name, pattern, debug=False):
    sum_time = 0.0
    nb_targets = 0
    for cmd, time in all_targets[:-2]: # Skip last target that is total time
        if pattern.match(cmd):
            sum_time += time
            nb_targets +=1
        elif debug:
            print cmd
    print("Sum time %s targets (%d): %f (%f%%)" % (name, nb_targets, sum_time, sum_time/total_make_time*100.0))

def AnalyzeCompilationTimes(log_file_name):
    print("File name: %s" % log_file_name)
    all_targets, skipped_lines = read_log_file(log_file_name)
    total_make_time = all_targets[-1][1]
    print("Total time (%s): %f" % (all_targets[-1][0], total_make_time))
    sum_time = 0.0
    for cmd, time in all_targets[:-2]: # Skip last target that is total time
        sum_time += time
    print("Sum time all targets: %f" % sum_time)
    # Find list commands used:
    set_commands = set()
    for cmd, time in all_targets[:-2]:
        cmd_split = cmd.split(" ")
        set_commands.add(cmd_split[0])
    print("All commands: %s" % str(set_commands))
    TargetTime(all_targets, total_make_time, 'swig', re.compile('.*swig.*bin.*swig.*python.*'))
    TargetTime(all_targets, total_make_time,'igenerator', re.compile('.*igenerator.py.*'))
    TargetTime(all_targets, total_make_time,'C++ Tests', re.compile('.*test.*Test\.cxx.*'))
    TargetTime(all_targets, total_make_time,'Link static libs', re.compile('.*/usr/bin/ar.*'))
    TargetTime(all_targets, total_make_time,'Compilation', re.compile('.*/usr/bin/c.*'))
    TargetTime(all_targets, total_make_time,'Compilation C', re.compile('.*/usr/bin/cc.*'))
    TargetTime(all_targets, total_make_time,'Compilation C++', re.compile('.*/usr/bin/c\+\+.*'))
    TargetTime(all_targets, total_make_time,'CMake', re.compile('.*/usr/local/bin/cmake.*'))
    TargetTime(all_targets, total_make_time,'Ranlib', re.compile('.*/usr/bin/ranlib.*'))
    TargetTime(all_targets, total_make_time,'castxml', re.compile('.*castxml/bin/castxml.*'))
    TargetTime(all_targets, total_make_time,'make', re.compile('.*make.*'))
    TargetTime(all_targets, total_make_time,'env', re.compile('.*env.*'))
    TargetTime(all_targets, total_make_time,'H5make_libsettings', re.compile('.*H5make_libsettings.*'))
    TargetTime(all_targets, total_make_time,'python', re.compile('.*bin/python.*'))
    TargetTime(all_targets, total_make_time,'H5detect', re.compile('.*H5detect.*'))
    TargetTime(all_targets, total_make_time,'itkmkg3states', re.compile('.*itkmkg3states.*'))
    TargetTime(all_targets, total_make_time,'Wrapping', re.compile('.*Wrapping.*'))
    TargetTime(all_targets, total_make_time,'HeaderTest', re.compile('.*HeaderTest.*'))

In [5]:
AnalyzeCompilationTimes("log_time_verbose_2digits")

File name: log_time_verbose_2digits
Skipping a line that has multiple ':' character
Skipping a line that has multiple ':' character
Number of lines: 190440
Number of skipped lines: 2
Number of targets: 8276
Total time ("make -j24"): 42432.598400
Sum time all targets: 41875.057500
All commands: set(['"../../Generators/CastXML/castxml/bin/castxml', '"make', '"/usr/bin/c++', '"/home/francois.budin/devel/itk/ITK/build/Wrapping/Generators/SwigInterface/swig/bin/swig', '"/usr/local/bin/cmake', '"env', '"/usr/bin/cc', '"/home/francois.budin/devel/itk/ITK/build/bin/H5make_libsettings"', '"/home/francois.budin/.virtualenvs/itk_dev/bin/python', '"/usr/bin/ranlib', '"make"', '"/usr/bin/ar', '"../../CastXML/castxml/bin/castxml', '"/home/francois.budin/devel/itk/ITK/build/bin/H5detect"', '"./itkmkg3states'])
Sum time swig targets (820): 12877.647300 (30.348477%)
Sum time igenerator targets (92): 760.875700 (1.793140%)
Sum time C++ Tests targets (1174): 9557.376700 (22.523666%)
Sum time Link static 

In [6]:
# This was run with `CTest` so total time includes configuration and testing time on top of compilation time.
AnalyzeCompilationTimes("log_time_testdrivers_not_compiled")

File name: log_time_testdrivers_not_compiled
Skipping a line that has multiple ':' character
Number of lines: 171305
Number of skipped lines: 1
Number of targets: 7448
Total time ("ctest -S my_dashboard.cmake -VV -R Python"): 31343.335000
Sum time all targets: 30539.744700
All commands: set(['"make', '"/usr/bin/c++', '"/home/francois.budin/devel/itk/MyTests/ITK-build/Modules/ThirdParty/TIFF/src/itktiff/itkmkg3states', '"/usr/local/bin/cmake', '"env', '"/usr/bin/cc', '"/home/francois.budin/devel/itk/MyTests/ITK-build/Wrapping/Generators/CastXML/castxml/bin/castxml', '"/home/francois.budin/devel/itk/MyTests/ITK-build/bin/H5detect"', '"/home/francois.budin/.virtualenvs/itk_dev/bin/python', '"/usr/bin/ranlib', '"/home/francois.budin/devel/itk/MyTests/ITK-build/Wrapping/Generators/SwigInterface/swig/bin/swig', '"make"', '"/usr/bin/ar', '"/home/francois.budin/devel/itk/MyTests/ITK-build/bin/H5make_libsettings"'])
Sum time swig targets (820): 14609.278100 (46.610477%)
Sum time igenerator targ

In [7]:
print('Speed up disabling TestDrivers compilation: %f' %(41875.057500-30539.744700) )

Speed up disabling TestDrivers compilation: 11335.312800
